In [1]:
import numpy as np
from my_utils import read_csv,get_pivot_data

In [2]:
from factor_gen.factor101 import get_raw_factor101

df = read_csv(dir_name='C:/Users/DELL/Desktop/recent/研一下/学习/量化/final_project/quant_code/raw_data/combine_data/',file_name='factor101_financial')

In [3]:
#https://www.joinquant.com/view/community/detail/b8ae4f443d688b7d479c07dffdf3fd56?type=1
time_data = get_pivot_data(df,'time')

def get_ic_beta_t(all_data_dict,factor_name):
    """
    输入:所有截面期的因子数据字典{'2020-01-31':因子df,...}
    输出:所有因子的ic,因子收益率,t值的df
    """
    # 定义存放所有期,所有因子的ic,beta,t字典
    factors_ic_all_period={}
    factors_beta_all_period = {}
    factors_t_all_period={}

    # 遍历因子数据每一期
    for k,v in all_data_dict.items():

        # 定义存放单期所有因子的ic列表
        factors_ic_period = []
        factors_beta_period = []
        factors_t_period = []
        
        # 遍历每列因子
        for factor in v.columns[:-1]:

            ########### 因子IC ############
            # 计算与最后一列的收益秩相关系数
            ic = st.spearmanr(v[factor],v['return'])[0]
            # 依次存入列表
            factors_ic_period.append(ic)

            ########### 因子收益率,t值 ###########
            # 每列因子与收益率RLM回归,得到系数,t值
            # 加截距,变成二维
            x=sm.add_constant(v[factor])
            model = sm.RLM(v['return'],x).fit()
            factors_beta_period.append(model.params[1])
            factors_t_period.append(model.tvalues[1])
        
        # 将因子列表存入这一期的字典
        factors_ic_all_period[k] = factors_ic_period
        factors_beta_all_period[k] = factors_beta_period 
        factors_t_all_period[k] = factors_t_period
        print(f"{k}数据已处理")
    #------------将得到的3个字典转为df--------------#
    ic_df = dict_to_df(factors_ic_all_period,factor_name,'IC')
    beta_df = dict_to_df(factors_beta_all_period,factor_name,'beta')
    t_df = dict_to_df(factors_t_all_period,factor_name,'t')
    print("所有数据处理完毕!")
    return ic_df,beta_df,t_df

NameError: name 'factor' is not defined

In [11]:
import scipy.stats as st

def pn_TransNorm_per_col(x):
    rank1 = x.rank(axis=0)
    rank2 = rank1.count()
    rank4 = (rank1 - 1) / rank2
    rank5 = rank4 + 1 / 2 / rank2
    return st.norm.ppf(rank5)

def get_per_day_return(per_day_data, factor, ret='y'):
    f = pn_TransNorm_per_col(per_day_data[factor])
    ret_pd = f * per_day_data[ret]
    ret_pd = ret_pd.mean()
    return ret_pd

def get_SharpRatio(total_data, date, fator, ret='y'):
    return_line = total_data.groupby([str(date)]).apply(lambda x : get_per_day_return(x, str(fator), str(ret)))
    sr = return_line.mean() / return_line.std() * 15
    return round(sr, 3), return_line

def PerformanceWithCost(f1, TotalRet, delayNum, cost, fig, SDate, EDate):
    # 1 normalize
    f1_stand = pn_TransNorm(f1.round(4))
    # 2 ts_Delay 2 days
    f1_stand_D2 = ts_Delay(f1_stand, delayNum)
    # get the factor return for each stocks
    factorRet = f1_stand_D2 * TotalRet
    # 3 convert negative factor to positive
    if factorRet.mean(1).mean() < 0:
        factorRet = factorRet * -1
    # 4 get cost
    Cost = GetCost(f1_stand_D2, cost)
    factorRet = factorRet - Cost
    # 5 choose date & clean data
    factorRet = factorRet.iloc[SDate:EDate]
    factorRetLine = factorRet.mean(1)
    factorRetLine[factorRetLine == 0] = np.nan
    # 6 get Sharpe ratio and annual return
    sr1 = factorRetLine.mean() / factorRetLine.std() * 15
    ret1 = factorRetLine.mean() * 250
    # 7 plot picture if fig == 1
    if fig == 1:
        factorRetLine[np.isnan(factorRetLine)] = 0
        Cumsums = factorRetLine.cumsum()
        plt.plot(Cumsums)
        plt.title('Factor Return: Sharpe Ratio = ' + str(round(sr1, 4)))
    # print
    print('SR:', round(sr1, 3), 'AR:', round(ret1, 3), 'TO:', GetTurnover(f1_stand))
    return round(sr1, 3), round(ret1, 3), factorRetLine

In [1]:
import pandas as pd
from utils import calculators
import factor_gen.y as y
from factor_gen.factor101 import get_raw_factor101
print('generate x...')
data = get_raw_factor101()
data[list(data.columns)[2:]] = data.groupby('Ticker').apply(lambda x:calculators.ts_Decay2(x[list(x.columns)[2:]],5)).reset_index(drop =True)
data['Ticker'] = data['Ticker'].astype(int)
print('generate y...')
y = y.get_day_rate()
y['rate'] = y['rate'].shift(-20)#后移20天（用X预测20天之后的Y）

total_data = pd.merge(data,y,how = 'inner',left_on = ['Ticker','PubDate'],right_on = ['code','time'])
total_data = total_data.sort_values(by =['PubDate'],ascending =True)

generate x...
finish preparing forecast data
finish preparing balance sheet data
finish preparing incomestatement data
finish merging 3 datas
generate y...
500
1000
1500
2000
2500
3000
3500
4000
4500


In [12]:
get_SharpRatio(total_data, 'PubDate', 'ROE', 'rate')

(0.102,
 PubDate
 2018-01-02    0.000000
 2018-01-03   -0.000890
 2018-01-04    0.004454
 2018-01-05   -0.009469
 2018-01-06    0.000500
                 ...   
 2022-04-02   -0.000564
 2022-04-03   -0.000637
 2022-04-04   -0.000824
 2022-04-05    0.000314
 2022-04-06    0.000000
 Length: 1556, dtype: float64)

In [3]:
import numpy as np
def get_factor_corr(total_data, code, f1, f2):
    corr = total_data.groupby([str(code)]).apply(lambda x: CorrRet(x[str(f1)], x[str(f2)]))
    return corr.mean()


def CorrRet(ret1, ret2):
    idx = ret1 != 0
    ret1_ = ret1[idx]
    ret2_ = ret2[idx]
    cor_ = np.corrcoef(ret1_, ret2_)
    cor_ = cor_[0, 1]
    return round(cor_, 4)

get_factor_corr(total_data, 'code', 'DILUTED_EPS', 'ROE')

0.8126875

In [2]:
total_data.tail(5)

,PubDate,Ticker,REVENUE,T_PROFIT,DILUTED_EPS,INCOME_TAX,N_PROFIT,REVENUE_GROWTH,T_PROFIT_GROWTH,ROE,...,EProfitCeiling,EProfitParentRateFloor,EProfitParentRateCeiling,EProfitParentFloor,EProfitParentCeiling,EEPSFloor,EEPSCeiling,time,rate,code
889264,2022-04-06,2120,1.943388e+09,1.560825e+08,0.3400,3.030150e+07,1.257810e+08,1.000000,1.000000,0.094250,...,NaN,570.77,608.66,177000000.0,187000000.0,NaN,NaN,2022-04-06,0.0,2120
564656,2022-04-06,889,3.167811e+10,3.988338e+09,0.8025,5.807514e+08,3.407587e+09,1.000000,1.000000,0.158607,...,NaN,116.60,141.80,736000000.0,822400000.0,NaN,NaN,2022-04-06,0.0,889
1960136,2022-04-06,2911,9.588322e+09,4.400670e+08,NaN,7.477157e+07,3.652954e+08,1.058682,0.838068,0.056907,...,NaN,823.00,933.00,250000000.0,280000000.0,NaN,NaN,2022-04-06,0.0,2911
680437,2022-04-06,998,1.266212e+09,9.319958e+07,0.2600,-1.382820e+07,1.070278e+08,1.000000,1.000000,0.088960,...,NaN,67.00,101.00,100000000.0,120000000.0,NaN,NaN,2022-04-06,0.0,998
1686384,2022-04-06,2696,8.754194e+08,5.472962e+07,0.3029,2.874453e+06,5.185517e+07,1.000000,1.000000,0.062544,...,NaN,0.00,50.00,8870000.0,13300000.0,0.048,0.072,2022-04-06,0.0,2696
